<a href="https://colab.research.google.com/github/dhnanjay/HuggingFace/blob/main/NER_extractions_using_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install py-llm-core
# https://github.com/advanced-stack/py-llm-core?tab=readme-ov-file

In [2]:
%mkdir -p ~/.cache/py-llm-core/models
%cd ~/.cache/py-llm-core/models
# !wget -O nuextract-tiny https://huggingface.co/advanced-stack/NuExtract-tiny-GGUF/resolve/main/nuextract-tiny-f16.gguf?download=true

/root/.cache/py-llm-core/models


UsageError: Line magic function `%wget` not found.


In [3]:
!wget -O nuextract-tiny https://huggingface.co/advanced-stack/NuExtract-tiny-GGUF/resolve/main/nuextract-tiny-f16.gguf?download=true

--2024-06-25 18:59:03--  https://huggingface.co/advanced-stack/NuExtract-tiny-GGUF/resolve/main/nuextract-tiny-f16.gguf?download=true
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.24, 18.172.134.4, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/ba/6f/ba6f9b7771682fa7863e549a2b715b9c972dcd0f9885ed9bbae6d127b8b71dd3/0a0b03c6139d2318bcaf824666968bcf7e7709c9bd7d2495f9b05e71cb8ce4b6?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27nuextract-tiny-f16.gguf%3B+filename%3D%22nuextract-tiny-f16.gguf%22%3B&Expires=1719601143&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxOTYwMTE0M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2JhLzZmL2JhNmY5Yjc3NzE2ODJmYTc4NjNlNTQ5YTJiNzE1YjljOTcyZGNkMGY5ODg1ZWQ5YmJhZTZkMTI3YjhiNzFkZDMvMGEwYjAzYzYxMzlkMjMxOGJjYWY

In [4]:
from dataclasses import dataclass
from llm_core.parsers import NuExtractParser

#: NuExtract model needs default values for the class fields
@dataclass
class Book:
    title: str = ""
    summary: str = ""
    author: str = ""
    published_year: int = ""


text = """Foundation is a science fiction novel by American writer
Isaac Asimov. It is the first published in his Foundation Trilogy (later
expanded into the Foundation series). Foundation is a cycle of five
interrelated short stories, first published as a single book by Gnome Press
in 1951. Collectively they tell the early story of the Foundation,
an institute founded by psychohistorian Hari Seldon to preserve the best
of galactic civilization after the collapse of the Galactic Empire.
"""

with NuExtractParser(Book) as parser:
    response = parser.parse(text)
    print(response)


Book(title='Foundation', summary='a science fiction novel by American writer Isaac Asimov. It is the first published in his Foundation Trilogy (later expanded into the Foundation series). Foundation is a cycle of five interrelated short stories, first published as a single book by Gnome Press in 1951.', author='Isaac Asimov', published_year='1951')


In [5]:
from dataclasses import dataclass
from llm_core.parsers import NuExtractParser

#: NuExtract model needs default values for the class fields
@dataclass
class Memo:
    Accounting_Subject: str = ""
    Company_Name: str = ""
    Date: str = ""


text = """
To:                      Memorandum to Accounting File
Prepared By:     CNM LLP, on behalf of Management
Reviewed By:    Juan Ojeda, Controller
Date:                  April 4, 2022
Subject:             ASC 606 – Revenue Recognition Standard Policy
Purpose:
The purpose of this memo is to analyze and document the accounting for revenue recognition under Financial Accounting Standards Board (“FASB”) Accounting Standards Codification 606, Revenue From Contracts With Customers, as amended (“ASC 606”), by Ecosense Lighting, Inc. (“the Company”).
Background:
In May 2014, the FASB issued Account Standards Update (“ASU”) 2014-09, which provided a new framework for revenue recognition. The core principles of ASC 606 state that an entity should recognize revenue to depict the transfer of promised goods or services to customers in an amount that reflects the consideration to which the entity expects to be entitled in exchange for those goods or services. The guidance provides a five-step analysis to achieve those core principles:
Identify the contract(s) with a customer
Identify the performance obligations in the contract
Determine the transaction price
Allocate the transaction price to the performance obligations in the contract
Recognize revenue when (or as) the entity satisfies a performance obligation
Subsequent to ASU 2014-09, the FASB issued several authoritative guidance updates that amended various aspects of the new standard to clarify certain terms, guidance, and disclosure requirements. In particular, the updates amended principal versus agent guidance, accounting for shipping and handling costs, disclosure requirements for remaining performance obligations, impairment testing for contract costs and accrual of advertising costs, as well as clarifies several previous examples.
ASC 606 was adopted by the Company with an effective date of January 1, 2019. As such, this technical analysis does not cover the Company’s initial adoption or implementation financial impact.
Company Overview and Revenue Background:
Ecosense Lighting was founded in 2009 and is an LED technology company. Ecosense is a team of LED lighting pioneers with over 100 years of combined[NP1]  experience and a genuine passion for delivering advanced solid-state lighting solutions. Ecosense delivers a broad line of high-quality, cutting-edge LED
  [NP1]Add this for clarity--please advise if you agree
"""

with NuExtractParser(Memo) as parser:
    response = parser.parse(text)
    print(response)

Memo(Accounting_Subject='Revenue Recognition Standard Policy', Company_Name='Ecosense Lighting, Inc.', Date='April 4, 2022')


In [9]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer


def predict_NuExtract(model,tokenizer,text, schema,example = ["","",""]):
    schema = json.dumps(json.loads(schema), indent=4)
    input_llm =  "<|input|>\n### Template:\n" +  schema + "\n"
    for i in example:
      if i != "":
          input_llm += "### Example:\n"+ json.dumps(json.loads(i), indent=4)+"\n"

    input_llm +=  "### Text:\n"+text +"\n<|output|>\n"
    input_ids = tokenizer(input_llm, return_tensors="pt",truncation = True, max_length = 4000)#.to("cuda")

    output = tokenizer.decode(model.generate(**input_ids)[0], skip_special_tokens=True)
    return output.split("<|output|>")[1].split("<|end-output|>")[0]


model = AutoModelForCausalLM.from_pretrained("numind/NuExtract-tiny", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("numind/NuExtract-tiny", trust_remote_code=True)

# model.to("cuda")

model.eval()




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1024, out_features=2816, bias=False)
          (up_proj): Linear(in_features=1024, out_features=2816, bias=False)
          (down_proj): Linear(in_features=2816, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Line

In [10]:
%%time

text = """We introduce Mistral 7B, a 7–billion-parameter language model engineered for
superior performance and efficiency. Mistral 7B outperforms the best open 13B
model (Llama 2) across all evaluated benchmarks, and the best released 34B
model (Llama 1) in reasoning, mathematics, and code generation. Our model
leverages grouped-query attention (GQA) for faster inference, coupled with sliding
window attention (SWA) to effectively handle sequences of arbitrary length with a
reduced inference cost. We also provide a model fine-tuned to follow instructions,
Mistral 7B – Instruct, that surpasses Llama 2 13B – chat model both on human and
automated benchmarks. Our models are released under the Apache 2.0 license.
Code: https://github.com/mistralai/mistral-src
Webpage: https://mistral.ai/news/announcing-mistral-7b/"""

schema = """{
    "Model": {
        "Name": "",
        "Number of parameters": "",
        "Number of max token": "",
        "Architecture": []
    },
    "Usage": {
        "Use case": [],
        "Licence": ""
    }
}"""

prediction = predict_NuExtract(model,tokenizer,text, schema,example = ["","",""])
print(prediction)

Setting `pad_token_id` to `eos_token_id`:151646 for open-end generation.



{
    "Model": {
        "Name": "Mistral 7B",
        "Number of parameters": "7 billion",
        "Number of max token": "",
        "Architecture": [
            "grouped-query attention (GQA)",
            "sliding window attention (SWA)"
        ]
    },
    "Usage": {
        "Use case": [
            "reasoning",
            "mathematics",
            "code generation"
        ],
        "Licence": "Apache 2.0"
    }
}

CPU times: user 33.4 s, sys: 417 ms, total: 33.8 s
Wall time: 36.9 s


In [15]:
%%time
text = """
To:                      Memorandum to Accounting File
Prepared By:     CNM LLP, on behalf of Management
Reviewed By:    Juan Ojeda, Controller
Date:                  April 4, 2022
Subject:             ASC 606 – Revenue Recognition Standard Policy
Purpose:
The purpose of this memo is to analyze and document the accounting for revenue recognition under Financial Accounting Standards Board (“FASB”) Accounting Standards Codification 606, Revenue From Contracts With Customers, as amended (“ASC 606”), by Ecosense Lighting, Inc. (“the Company”).
Background:
In May 2014, the FASB issued Account Standards Update (“ASU”) 2014-09, which provided a new framework for revenue recognition. The core principles of ASC 606 state that an entity should recognize revenue to depict the transfer of promised goods or services to customers in an amount that reflects the consideration to which the entity expects to be entitled in exchange for those goods or services. The guidance provides a five-step analysis to achieve those core principles:
Identify the contract(s) with a customer
Identify the performance obligations in the contract
Determine the transaction price
Allocate the transaction price to the performance obligations in the contract
Recognize revenue when (or as) the entity satisfies a performance obligation
Subsequent to ASU 2014-09, the FASB issued several authoritative guidance updates that amended various aspects of the new standard to clarify certain terms, guidance, and disclosure requirements. In particular, the updates amended principal versus agent guidance, accounting for shipping and handling costs, disclosure requirements for remaining performance obligations, impairment testing for contract costs and accrual of advertising costs, as well as clarifies several previous examples.
ASC 606 was adopted by the Company with an effective date of January 1, 2019. As such, this technical analysis does not cover the Company’s initial adoption or implementation financial impact.
Company Overview and Revenue Background:
Ecosense Lighting was founded in 2009 and is an LED technology company. Ecosense is a team of LED lighting pioneers with over 100 years of combined[NP1]  experience and a genuine passion for delivering advanced solid-state lighting solutions. Ecosense delivers a broad line of high-quality, cutting-edge LED
  [NP1]Add this for clarity--please advise if you agree
"""

# schema = """{
#     "Accounting Memo": {
#         "Accounting Subject": "",
#         "Company Name": "",
#         "Date": "",
#     },
# }"""

schema = """{

    "Accounting Memo": {

        "Accounting Subject": "",

        "Company Name": "",

        "Date": ""

    }

}"""

prediction = predict_NuExtract(model,tokenizer,text, schema,example = ["","",""])
print(prediction)



Setting `pad_token_id` to `eos_token_id`:151646 for open-end generation.



{
    "Accounting Memo": {
        "Accounting Subject": "ASC 606 \u2013 Revenue Recognition Standard Policy",
        "Company Name": "Ecosense Lighting, Inc.",
        "Date": "April 4, 2022"
    }
}

CPU times: user 29.5 s, sys: 472 ms, total: 30 s
Wall time: 32.3 s


In [13]:
!pip show transformers

Name: transformers
Version: 4.41.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [ ]:
 "Purpose": "",

        "Simple or Policy Memo": "",
